## **Max covering problem**

build a given number of facilities to cover as many demands as possible.

 ***MIP Model***

***Indices and Parameters***

- $I = \{1, 2, ..., m\}$: A set of demand points.
- $J = \{1, 2, ..., n\}$: A set of potential facility locations.
- $d_{ij}$: Distance from demand point $i$ to location $j$, for $i \in I, j \in J$, where $d_{ij} > 0$.
- $p$: max number of facilities allowed to build.
- $a_{ij}$: Binary parameter, where $a_{ij} = 1$ if $d_{ij} < s$ (i.e., demand $i$ can be covered by location $j$), or $0$ otherwise.

***Decision Variables***

- $x_j$: Binary variable that equals 1 if a facility is allocated at location $j$ and 0 otherwise, for $j \in J$.
- $y_i$: Binary variable that equals 1 demand $i$ is covered by any facility. otherwise 0.

***Objective Function***

Maximize the number of covered demand points:

$$ \text{Maximize } \sum_{i \in I} y_i $$


***Constraints***

1. coverage constraint: Ensure $y_i$ is 1 if demand $i$ is covered by at least one located facility:

$$ y_i \leq \sum_{j \in J} a_{ij} x_j, \quad \forall i \in I $$

This constraint guarantees that if demand point $i$ is covered by any facility within the service level, $y_i$ should be 1. It effectively ties the coverage decision to the actual placement of facilities and their ability to cover demand points within the service level.

2. Facility Limitation: Limit the number of facilities that can be located:

$$ \sum_{j \in J} x_j \leq p $$

This constraint restricts the total number of facilities to be located to $p$, ensuring the solution adheres to the maximum allowed facility placements.


***Code***

In [ ]:
import pulp

class MaxCoveringProblem:
    def __init__(self, I, J, d, s, p):
        """
        Initializes the Max Covering Problem model.

        Parameters:
        - I: Set of demand points.
        - J: Set of potential facility locations.
        - d: Dictionary with keys (i, j) representing distances from demand i to location j.
        - s: Service level, a maximum distance for demands to be considered covered.
        - p: Max number of facilities allowed to build
        """
        self.I = I
        self.J = J
        self.d = d
        self.s = s
        self.p = p 

        #creating a_ij
        self.a = {(i, j): 1 if d[(i, j)] < s else 0 for i in I for j in J}

        #initialize the problem
        self.problem = pulp.LpProblem("MaxCoveringProblem", pulp.LpMaximize)

        #DVs
        self.x = pulp.LpVariable.dicts("x", J, cat=pulp.LpBinary)
        self.y = pulp.LpVariable.dicts("y", I, cat=pulp.LpBinary)

    def build_model(self):
        """
        Builds the MIP model for the max covering problem.
        """
        #obj
        self.problem += pulp.lpSum(self.y[i] for i in self.I)

        # constraints
        #1. coverage constraint: Ensure y_i is 1 if demand i is covered by at least one located facility:
        for i in self.I:
            self.problem += pulp.lpSum(self.a[(i, j)] * self.x[j] for j in self.J) >= self.y[i]

        #2. Facility Limitation: Limit the number of facilities that can be located:
        self.problem += pulp.lpSum(self.x[j] for j in self.J) <= self.p
    
    def solve(self):
        result_status = self.problem.solve()
        
        # Check if the solution is optimal
        if pulp.LpStatus[result_status] == "Optimal":
            print("Status:", pulp.LpStatus[result_status])

            # Print the solution
            for j in self.J:
                if self.x[j].value() == 1:
                    print(f"Facility at location {j} is allocated.")
            for i in self.I:
                if self.y[i].value() == 1:
                    print(f"demand {i} is satisfied.")
        else:
            print("No optimal solution found. Status:", pulp.LpStatus[result_status])





Example:

In [ ]:
# Example parameters
I = [1, 2, 3, 4, 5]
J = [1, 2, 3, 4]
d = {
    (1, 1): 8, (1, 2): 10, (1, 3): 15, (1, 4): 16,
    (2, 1): 12, (2, 2): 2, (2, 3): 4, (2, 4): 14,
    (3, 1): 14, (3, 2): 5, (3, 3): 8, (3, 4): 7,
    (4, 1): 5, (4, 2): 8, (4, 3): 10, (4, 4): 6,
    (5, 1): 10, (5, 2): 15, (5, 3): 6, (5, 4): 9,
}
s = 10
p = 2

problem = MaxCoveringProblem(I,J,d,s,p)
problem.build_model()
problem.solve()